[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/utkarshp1161/Active-learning-in-microscopy/blob/main/notebooks/MultiFidelity(MultiTask)_GPyTorch.ipynb)
# MultiFidelity(MultiTask) GP tutorial using GPyTorch 

This notebook demonstrates implementation of multitask GP in Gpytorch

- Rewritten in Gpytorch by [Utkarsh Pratiush](https://github.com/utkarshp1161). Inspired from [Original implementation in Gpax](https://github.com/SergeiVKalinin/BO_Research/blob/master/MultiTaskGP_tutorial.ipynb) by [Maxim Ziatdinov](https://github.com/ziatdinovmax) and [SVK](https://github.com/SergeiVKalinin).
- For reading related to this notebook please refer to paper by [Edwin et al 2007](https://www.google.com/search?q=Multi-task+Gaussian+Process+Prediction&oq=Multi-task+Gaussian+Process+Prediction&gs_lcrp=EgZjaHJvbWUyBggAEEUYOTIGCAEQRRg8MgYIAhBFGD0yBggDEEUYPTIGCAQQLhhA0gEHMjU0ajBqMagCALACAA&sourceid=chrome&ie=UTF-8)
- Reference to relevant [Gpytorch example](https://github.com/cornellius-gp/gpytorch/blob/main/examples/03_Multitask_Exact_GPs/Multitask_GP_Regression.ipynb)

Warning: Though Multitask and Multifidelity are fundamentally different methods we are using the term interchangeably here.


## 1a. Install modules 

In [1]:
#install
!pip install -q botorch==0.12.0
!pip install -q gpytorch==1.13

import math
import torch
import gpytorch
import numpy as np
from matplotlib import pyplot as plt

## 1b. Ground truth function

In [ ]:
# Create toy datasets
def theoretical_signal(x):
    return 2 * np.sin(x/10) + 0.5 * np.sin(x/2) + 0.1 * x

np.random.seed(0)

# Fidelity 1 - "theoretical model"
X1 = np.linspace(0, 100, 100)
y1 = theoretical_signal(X1)

# Fidelity 2 - "experimental measurements"
X2 = np.concatenate([np.linspace(0, 25, 25), np.linspace(75, 100, 25)])
y2 = 1.5 * theoretical_signal(X2) - 5 + np.random.normal(0, 0.5, X2.shape) + np.sin(X2/15)

# Ground truth for Fidelity 2
X_full_range = np.linspace(0, 100, 200)
y2_true = 1.5 * theoretical_signal(X_full_range) - 5 + np.sin(X_full_range/15)

# Plot the initial data
plt.figure(figsize=(10, 6))
plt.plot(X1, y1, 'b-', label='Theoretical Model (Fidelity 1)', alpha=0.6)
plt.scatter(X2, y2, c='k', label='Experimental Data (Fidelity 2)', alpha=0.6)
plt.plot(X_full_range, y2_true, 'k--', label='True function (Fidelity 2)', linewidth=2)
plt.xlabel('Frequency')
plt.ylabel('Signal Strength')
plt.legend()
plt.grid(True)
plt.title('Initial Data')
plt.show()

## 1c Regular GP for experimental data alone:

### 1c. i) Define the model

In [3]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
    
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

### 1c. ii) sample data and set optimizer

In [20]:
# Convert X2 and y2 to PyTorch tensors
train_x = torch.from_numpy(X2).float()
train_y = torch.from_numpy(y2).float()

# Initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)


# Training
model.train()
likelihood.train()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

training_iterations = 100

### 1c. iii) Training loop

In [ ]:
for i in range(training_iterations):
    optimizer.zero_grad()
    output = model(train_x)
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
    optimizer.step()

### 1c. iv) Posterior plot

In [ ]:
# Set the model in evaluation mode
model.eval()
likelihood.eval()

# Create test points
test_x = torch.linspace(0, 100, 200)

# Get posterior distribution
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    observed_pred = likelihood(model(test_x))
    mean = observed_pred.mean
    lower, upper = observed_pred.confidence_region()

# Convert to numpy for plotting
test_x_np = test_x.numpy()
mean_np = mean.numpy()
lower_np = lower.numpy()
upper_np = upper.numpy()

# Create the plot
plt.figure(figsize=(10, 6))
plt.scatter(X2, y2, c='k', label='Training Data (Fidelity 2)', alpha=0.6)
plt.plot(X1, y1, 'b-', label='Theoretical Model (Fidelity 1)', alpha=0.4)
plt.plot(X_full_range, y2_true, 'k--', label='True Function (Fidelity 2)', alpha=0.6)
plt.plot(test_x_np, mean_np, 'r-', label='Posterior Mean')
plt.fill_between(test_x_np, lower_np, upper_np, alpha=0.3, color='red')
plt.xlabel('Frequency')
plt.ylabel('Signal Strength')
plt.legend()
plt.grid(True)
plt.title('GP Posterior Distribution')
plt.show()


## 1d. MultitaskGP 

### 1d. i) Define the model

In [9]:
class MultiFidelityGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, num_latent):
        super(MultiFidelityGPModel, self).__init__(train_x, train_y, likelihood)
        
        # Mean module
        self.mean_module = gpytorch.means.ConstantMean()
        
        # Base kernel for the data
        self.base_covar = gpytorch.kernels.MaternKernel(ard_num_dims=1)
        
        # Kernel for fidelity interactions with configurable rank
        self.task_covar = gpytorch.kernels.IndexKernel(num_tasks=2, rank=num_latent)
        
        # Scale the output
        self.scaling = gpytorch.kernels.ScaleKernel(self.base_covar)

    def forward(self, x):
        # Split input into coordinates and task indicators
        coordinates = x[..., 0:1]
        task_indicators = x[..., 1].long()
        
        mean_x = self.mean_module(coordinates)
        covar_x = self.scaling(coordinates)
        covar_i = self.task_covar(task_indicators)
        covar = covar_x.mul(covar_i)
        
        return gpytorch.distributions.MultivariateNormal(mean_x, covar)



### 1d. ii) Define the training loop

In [12]:
def train_and_predict(num_latent, training_iterations=50):
    # Initialize likelihood and model
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = MultiFidelityGPModel(train_x, train_y, likelihood, num_latent)

    # Training
    model.train()
    likelihood.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    for i in range(training_iterations):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
        optimizer.step()

    # Make predictions
    model.eval()
    likelihood.eval()

    # Create test points for fidelity 2
    test_x = torch.from_numpy(np.column_stack((X_full_range, np.ones_like(X_full_range)))).float()

    # Predict
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        predictions = likelihood(model(test_x))
        mean = predictions.mean
        lower, upper = predictions.confidence_region()

    # Plot the results
    plt.figure(figsize=(10, 6))
    plt.plot(X_full_range, mean.numpy(), 'r-', label='Multi-fidelity GP prediction')
    plt.fill_between(X_full_range, lower.numpy(), upper.numpy(), alpha=0.3, color='red')
    plt.plot(X_full_range, y2_true, 'k--', label='Ground Truth')
    plt.plot(X1, y1, 'b-', label='Theoretical Model (Fidelity 1)', alpha=0.6)
    plt.scatter(X2, y2, c='k', label='Experimental Data (Fidelity 2)', alpha=0.6)
    plt.xlabel('Frequency')
    plt.ylabel('Signal Strength')
    plt.legend()
    plt.grid(True)
    plt.title(f'Multi-fidelity GP Prediction (num_latent={num_latent})')
    plt.show()

    return model, likelihood


### 1d. iii) Choose parameters and run experiment - FOR num_latents = 1

In [ ]:
X1_with_fidelity = np.column_stack([X1, np.zeros_like(X1)])  # Fidelity 1
X2_with_fidelity = np.column_stack([X2, np.ones_like(X2)])   # Fidelity 2

# Concatenate both fidelities
train_x = torch.from_numpy(np.vstack([X1_with_fidelity, X2_with_fidelity])).float()
train_y = torch.from_numpy(np.concatenate([y1, y2])).float()

# Example usage with different number of latent dimensions
num_latent = 1  # You can change this value
model, likelihood = train_and_predict(num_latent, training_iterations = 500)

### 1d. iv) Choose parameters and run experiment - FOR num_latents = 2

In [ ]:
num_latent = 2  # You can change this value
model, likelihood = train_and_predict(num_latent, training_iterations = 500)